# Web scrapping multiple tests

In [1]:
# Import required libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [39]:
#Function to send an HTTP request and parse the HTML content of the page
def fetch_html(url):
    response = requests.get(url)  # Send an HTTP GET request to the URL
    if response.status_code == 200:  # Check if the request was successful (status code 200)
        # Parse the response content with BeautifulSoup and return the parsed HTML
        return BeautifulSoup(response.content, 'lxml')
    else:
        return None

In [7]:
url = "https://myanimelist.net/topanime.php"  # Define the target URL
soup = fetch_html(url)  # Fetch and parse the HTML content from the URL

# Scrapping Shingeki no Kyojin

## Anime Ranking 

In [36]:
soup.find('span',class_="lightLink top-anime-rank-text rank1").text

'1'

## Anime Score

In [38]:
soup.find('span',class_="text on score-label score-9").text

'9.13'

### Anime name

In [23]:
soup.find('div',class_="di-ib clearfix").a.text

'Shingeki no Kyojin: The Final Season - Kanketsu-hen'

### Anime URL

In [26]:
soup.find('div', class_="di-ib clearfix").a['href']

'https://myanimelist.net/anime/51535/Shingeki_no_Kyojin__The_Final_Season_-_Kanketsu-hen'

### Number of episodes

In [29]:
soup.find('div',class_="information di-ib mt4").text.split('\n')[1].strip()

'Special (2 eps)'

### Emission date

In [32]:
soup.find('div',class_="information di-ib mt4").text.split('\n')[2].strip()

'Mar 2023 - 2023'

### Number of members

In [33]:
soup.find('div',class_="information di-ib mt4").text.split('\n')[3].strip()

'369,372 members'